In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
import os
os.chdir('./drive/My Drive/TESI/ST-ResNet')

Mounted at /content/drive/


In [2]:
from __future__ import print_function
import os
import pickle
import numpy as np
import time
import h5py
import math

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from deepst.models.STResNet import stresnet
import deepst.metrics as metrics
from deepst.datasets import TaxiNYC

In [3]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.4.3
2.3.0
Num GPUs Available:  1


In [4]:
np.random.seed(1337)  # for reproducibility
tf.random.set_seed(2)
# parameters
DATAPATH = '../data'  # data path, you may set your own data path with the global envirmental variable DATAPATH
CACHEDATA = True  # cache data or NOT
path_cache = os.path.join(DATAPATH, 'CACHE', 'ST-ResNet')  # cache path
nb_epoch = 100  # number of epoch at training stage
# nb_epoch_cont = 100  # number of epoch at training (cont) stage
batch_size = 64  # batch size

T = 24  # number of time intervals in one day
lr = 0.0002  # learning rate
len_closeness = 3  # length of closeness dependent sequence
len_period = 1  # length of peroid dependent sequence
len_trend = 1  # length of trend dependent sequence
nb_residual_unit = 4 # paper says 4 for BikeNYC

nb_flow = 2  # there are two types of flows: inflow and outflow

days_test = 7*4
len_test = T * days_test
map_height, map_width = 16, 8  # grid size
path_result = 'RET'
path_model = 'MODEL'

if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)
if CACHEDATA and os.path.isdir(path_cache) is False:
    os.mkdir(path_cache)

In [5]:
def build_model(external_dim):
    c_conf = (len_closeness, nb_flow, map_height,
              map_width) if len_closeness > 0 else None
    p_conf = (len_period, nb_flow, map_height,
              map_width) if len_period > 0 else None
    t_conf = (len_trend, nb_flow, map_height,
              map_width) if len_trend > 0 else None
    model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                     external_dim=external_dim, nb_residual_unit=nb_residual_unit)
    adam = Adam(lr=lr)
    model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse])
    # model.summary()
    # from keras.utils.visualize_util import plot
    # plot(model, to_file='model.png', show_shapes=True)
    return model

In [6]:
def read_cache(fname):
    mmn = pickle.load(open('preprocessing_taxinyc.pkl', 'rb'))

    f = h5py.File(fname, 'r')
    num = int(f['num'].value)
    X_train, Y_train, X_test, Y_test = [], [], [], []
    for i in range(num):
        X_train.append(f['X_train_%i' % i].value)
        X_test.append(f['X_test_%i' % i].value)
    Y_train = f['Y_train'].value
    Y_test = f['Y_test'].value
    external_dim = f['external_dim'].value
    timestamp_train = f['T_train'].value
    timestamp_test = f['T_test'].value
    f.close()

    return X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test

In [7]:
def cache(fname, X_train, Y_train, X_test, Y_test, external_dim, timestamp_train, timestamp_test):
    h5 = h5py.File(fname, 'w')
    h5.create_dataset('num', data=len(X_train))

    for i, data in enumerate(X_train):
        h5.create_dataset('X_train_%i' % i, data=data)
    # for i, data in enumerate(Y_train):
    for i, data in enumerate(X_test):
        h5.create_dataset('X_test_%i' % i, data=data)
    h5.create_dataset('Y_train', data=Y_train)
    h5.create_dataset('Y_test', data=Y_test)
    external_dim = -1 if external_dim is None else int(external_dim)
    h5.create_dataset('external_dim', data=external_dim)
    h5.create_dataset('T_train', data=timestamp_train)
    h5.create_dataset('T_test', data=timestamp_test)
    h5.close()

In [8]:
print("loading data...")
ts = time.time()
fname = os.path.join(path_cache, 'TaxiNYC_C{}_P{}_T{}.h5'.format(
    len_closeness, len_period, len_trend))
if os.path.exists(fname) and CACHEDATA:
    X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = read_cache(
        fname)
    print("load %s successfully" % fname)
else:
    X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = TaxiNYC.load_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        preprocess_name='preprocessing_taxinyc.pkl', meta_data=True, datapath=DATAPATH)
    if CACHEDATA:
        cache(fname, X_train, Y_train, X_test, Y_test,
              external_dim, timestamp_train, timestamp_test)

print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])
print("\nelapsed time (loading data): %.3f seconds\n" % (time.time() - ts))

print('=' * 10)

loading data...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # Remove the CWD from sys.path while we load stuff.


load ../data/CACHE/ST-ResNet/TaxiNYC_C3_P1_T1.h5 successfully

 days (test):  [b'20141204', b'20141205', b'20141206', b'20141207', b'20141208', b'20141209', b'20141210', b'20141211', b'20141212', b'20141213', b'20141214', b'20141215', b'20141216', b'20141217', b'20141218', b'20141219', b'20141220', b'20141221', b'20141222', b'20141223', b'20141224', b'20141225', b'20141226', b'20141227', b'20141228', b'20141229', b'20141230', b'20141231']

elapsed time (loading data): 3.090 seconds



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


In [ ]:
for i in range(4):
    print(X_train[i].shape)

(40176, 6, 16, 8)
(40176, 2, 16, 8)
(40176, 2, 16, 8)
(40176, 8)


In [9]:
print("compiling model...")
ts = time.time()
model = build_model(external_dim)
hyperparams_name = 'TaxiNYC.c{}.p{}.t{}.resunit{}.lr{}'.format(
    len_closeness, len_period, len_trend, nb_residual_unit, lr)
fname_param = os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name))

early_stopping = EarlyStopping(monitor='val_rmse', patience=2, mode='min')
model_checkpoint = ModelCheckpoint(
    fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

print("\nelapsed time (compiling model): %.3f seconds\n" %
      (time.time() - ts))

print('=' * 10)

compiling model...
(None, 2, 16, 8)
(None, 2, 16, 8)
(None, 2, 16, 8)

elapsed time (compiling model): 5.611 seconds



In [10]:
print("training model...")
ts = time.time()
# model.load_weights('./MODEL/BikeNYC.c3.p1.t1.resunit4.lr0.0002.best.h5')
history = model.fit(X_train, Y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[model_checkpoint],
                    # callbacks=[early_stopping, model_checkpoint],
                    verbose=1)
model.save_weights(os.path.join(
    'MODEL', '{}.h5'.format(hyperparams_name)), overwrite=True)
pickle.dump((history.history), open(os.path.join(
    path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))
print("\nelapsed time (training): %.3f seconds\n" % (time.time() - ts))

training model...
Epoch 1/100
565/565 [==============================] - 13s 22ms/step - loss: 0.0062 - rmse: 0.0375 - val_loss: 0.0013 - val_rmse: 0.0364
Epoch 2/100
565/565 [==============================] - 12s 21ms/step - loss: 1.8052e-04 - rmse: 0.0133 - val_loss: 1.2796e-04 - val_rmse: 0.0111
Epoch 3/100
565/565 [==============================] - 12s 21ms/step - loss: 1.1723e-04 - rmse: 0.0108 - val_loss: 9.5570e-05 - val_rmse: 0.0096
Epoch 4/100
565/565 [==============================] - 12s 21ms/step - loss: 9.6255e-05 - rmse: 0.0097 - val_loss: 8.9297e-05 - val_rmse: 0.0092
Epoch 5/100
565/565 [==============================] - 12s 21ms/step - loss: 7.8251e-05 - rmse: 0.0088 - val_loss: 6.2743e-05 - val_rmse: 0.0077
Epoch 6/100
565/565 [==============================] - 12s 21ms/step - loss: 7.2577e-05 - rmse: 0.0084 - val_loss: 5.7898e-05 - val_rmse: 0.0074
Epoch 7/100
565/565 [==============================] - 12s 21ms/step - loss: 6.5726e-05 - rmse: 0.0080 - val_loss: 5.791

In [12]:
print('evaluating using the model that has the best loss on the valid set')

model.load_weights(fname_param)

score = model.evaluate(
    X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
      (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. ))

evaluating using the model that has the best loss on the valid set
Test score: 0.000026 rmse (norm): 0.005067 rmse (real): 43.273259
